In [70]:
import re
import pandas as pd
import numpy as np
import math

### Day 1

In [ ]:
f = open('../data/day_1_input.txt', 'r')

sum = 0

for line in f:
    nums = re.findall(r'\d', line)
    sum = sum + int(str(nums[0])+str(nums[-1]))

sum

In [ ]:
f = open('../data/day_1_input.txt', 'r')

sum = 0

digit_dict = {
    'one':1,
    'two':2,
    'three':3,
    'four':4,
    'five':5,
    'six':6,
    'seven':7,
    'eight':8,
    'nine':9
    }

for line in f:
    nums = re.findall(r'(?=(\d|one|two|three|four|five|six|seven|eight|nine))', line)
    for value in range(len(nums)):
        if nums[value] in digit_dict.keys():
            nums[value] = digit_dict[nums[value]]
    sum = sum + int(str(nums[0])+str(nums[-1]))

sum

### Day 2

In [ ]:
df = pd.DataFrame(columns = ['game_id', 'draw_id', 'red', 'blue', 'green'])

In [ ]:
f = open('../data/day_2_input.txt', 'r')

df = pd.DataFrame(columns = ['game_id', 'draw_id', 'red', 'blue', 'green'])

for line in f: 
    
    game_dict = {}

    colon = re.search(':', line).start()
    game_id = int(re.findall(r'\d+',line[:colon])[0])
    game_dict['game_id'] = game_id

    match = re.finditer(';', line)
    draws = [colon] + [m.start() for m in match] + [len(line)]

    for d in range(len(draws)-1):
        draw_str = line[draws[d]+1:draws[d+1]] 
        draw_id = d
        game_dict['draw_id'] = d

        colors = re.findall(r'blue|green|red', draw_str)
        scores = re.findall(r'\d+', draw_str)
        scores = [int(s) for s in scores]

        draw_dict = dict(zip(colors, scores))

        record_dict = game_dict.copy()
        record_dict.update(draw_dict)
        record_dict = {k:[record_dict[k]] for k in record_dict.keys()}
        
        df = pd.concat([df, pd.DataFrame.from_records(record_dict)])

final_df = (
    df
    .fillna(0)
    .assign(is_possible = lambda x: np.where((x['red']<=12) & (x['green']<=13) & (x['blue'] <= 14), 1, 0))
    .groupby('game_id')
    .agg({'is_possible':'min'})
    .reset_index()
    .loc[lambda x: x['is_possible']==1, :]
)

final_df['game_id'].sum()

In [ ]:
final_df = (
    df
    .fillna(0)
    .groupby('game_id')
    .max()
    .reset_index()
    .assign(power = lambda x: x['red']*x['blue']*x['green'])
)

final_df['power'].sum()

### Day 3

In [ ]:
f = open('../data/day_3_input.txt', 'r')

symbol_dict = {}
index = 0

for line in f:
    match_symbol = re.finditer(r'[^A-Za-z0-9.\n]', line)
    symbol_dict[index] = [m.start() for m in match_symbol]
    index = index + 1


In [ ]:
f = open('../data/day_3_input.txt', 'r')

gear_dict = {}
index = 0

for line in f:
    match_symbol = re.finditer(r'\*', line)
    gear_dict[index] = [m.start() for m in match_symbol]
    index = index + 1

gear_dict

In [ ]:
f = open('../data/day_3_input.txt', 'r')

index = 0
part_sum = 0

for line in f:
    match_part = re.finditer(r'\d+', line)
    parts = [(m.start(), m.end()) for m in match_part]
    for p in parts:
        is_valid = 0
        value = int(line[p[0]:p[1]])
        if p[0] - 1 in symbol_dict[index]:
            is_valid = 1
        if p[1] in symbol_dict[index]:
            is_valid = 1
        if index < 139:
            for x in symbol_dict[index + 1]:
                if x >= p[0] - 1 and x <= p[1]:
                    is_valid = 1
        if index > 1:
            for x in symbol_dict[index - 1]:
                if x >= p[0] - 1 and x <= p[1]:
                    is_valid = 1
        if is_valid == 1:
            part_sum = part_sum + value
    index = index + 1

part_sum

In [ ]:
f = open('../data/day_3_input.txt', 'r')

part_dict = {}
index = 0

for line in f:
    match_symbol = re.finditer(r'\d+', line)
    part_dict[index] = [(m.start(), m.end(), int(line[m.start():m.end()])) for m in match_symbol]
    index = index + 1

In [ ]:
f = open('../data/day_3_input.txt', 'r')

gear_dict = {}
index = 0

for line in f:
    match_symbol = re.finditer(r'\*', line)
    gear_dict[index] = [m.start() for m in match_symbol]
    index = index + 1

In [ ]:
gear_sum = 0

for line_index in range(139):
    for i in range(len(gear_dict[line_index])):
        matches = 0
        match_prod = 1
        # match_left
        end_index = [m[1] for m in part_dict[line_index]]
        if gear_dict[line_index][i] in end_index:
            part_value = part_dict[line_index][end_index.index(gear_dict[line_index][i])][2]
            matches = matches + 1
            match_prod = match_prod*part_value
        # match_right
        start_index = [m[0] for m in part_dict[line_index]]
        if gear_dict[line_index][i] + 1 in start_index:
            part_value = part_dict[line_index][start_index.index(gear_dict[line_index][i] + 1)][2]
            matches = matches + 1
            match_prod = match_prod*part_value
        # matches_above
        for m in part_dict[line_index - 1]:
            if not (m[0] >= gear_dict[line_index][i] + 2 or m[1] <= gear_dict[line_index][i] - 1):
                matches = matches + 1
                match_prod = match_prod*m[2]

        # matches_below
        for m in part_dict[line_index + 1]:
            if not (m[0] >= gear_dict[line_index][i] + 2 or m[1] <= gear_dict[line_index][i] - 1):
                matches = matches + 1
                match_prod = match_prod*m[2]

        if matches == 2:
            gear_sum = gear_sum + match_prod

gear_sum


### Day 4

In [ ]:
f = open('../data/day_4_input.txt', 'r')

total_score = 0

for line in f:

    winners = [int(x) for x in re.findall(r'\d+', line[re.search(":", line).end():re.search("\|", line).start()])]
    candidates = [int(x) for x in re.findall(r'\d+', line[re.search("\|", line).end():])]

    matches = 0
    for x in candidates:
        if x in winners:
            matches = matches + 1

    if matches == 0:
        points = 0
    else:
        points = 2**(matches - 1)

    total_score = total_score + points

total_score

In [ ]:
f = open('../data/day_4_input.txt', 'r')

total_score = 0
card_dict = {}

for line in f:

    card_id = int(re.findall(r'\d+', line[:re.search(":", line).start()])[0])
    winners = [int(x) for x in re.findall(r'\d+', line[re.search(":", line).end():re.search("\|", line).start()])]
    candidates = [int(x) for x in re.findall(r'\d+', line[re.search("\|", line).end():])]

    matches = 0
    for x in candidates:
        if x in winners:
            matches = matches + 1

    new_cards = [x+1 for x in range(card_id, card_id + matches)]
    card_dict[card_id] = new_cards

card_list = {x + 1:1 for x in range(206)}

for x in card_list.keys():
    for num in card_list.keys():
        if num in card_dict[x]:
            card_list[num] = card_list[num] + card_list[x]

sum(card_list.values())

### Day 5

In [104]:
f = open('../data/day_5_input.txt', 'r').read()

def data_map(input, map_df):
    for i in range(map_df.shape[0]):
        if input in range(map_df[i][1], map_df[i][1]+map_df[i][2]):
            return int(map_df[i][0] + input - map_df[i][1])
    return int(input)

cats = [
    'seeds', 
    'seed-to-soil map',
    'soil-to-fertilizer map', 
    'fertilizer-to-water map', 
    'water-to-light map', 
    'light-to-temperature map', 
    'temperature-to-humidity map',
    'humidity-to-location map'
    ]

data_dict = {}

for i in range(8):
    if i == 0:
        data_dict[cats[i]] =  [int(x) for x in re.findall(r'\d+', f[re.search(cats[i], f).end():re.search(cats[i+1], f).start()])]
    else:
        if i < 7:
            data_dict[cats[i]] = np.array([int(x) for x in re.findall(r'\d+', f[re.search(cats[i], f).end():re.search(cats[i+1], f).start()])]).reshape((-1, 3))
        else:
            data_dict[cats[i]] = np.array([int(x) for x in re.findall(r'\d+', f[re.search(cats[i], f).end():])]).reshape((-1, 3))


In [105]:
min_output = 999999999999999999999999

for i in range(len(data_dict['seeds'])):

    input = data_dict['seeds'][i]

    for x in [x for x in data_dict.keys() if x != 'seeds']:
        output = data_map(input, data_dict[x])
        input = output

    if output < min_output:
        min_output = output

print(min_output)


227653707


In [106]:
seed_list = []
for i in range(int(len(data_dict['seeds'])/2)):
    seed_list.append(range(data_dict['seeds'][2*i], data_dict['seeds'][2*i] + data_dict['seeds'][2*i + 1]))

In [122]:
# Returning a minimum with a huge step size over all possible seeds. 
# In this case, the square root of the length of each range
min_output = 999999999999999999999999
min_seed = 999999999999999999999999

for i in range(len(seed_list)):
    for j in range(0, len(seed_list[i]), math.floor(math.sqrt(len(seed_list[i])))):
        input = seed_list[i][j]

        for x in [x for x in data_dict.keys() if x != 'seeds']:
            output = data_map(input, data_dict[x])
            input = output

        if output < min_output:
            min_i = i
            min_j = j
            min_seed = seed_list[i][j]
            min_output = output

print('i: ', min_i)
print('j: ', min_j)
print('seed: ', min_seed)
print('output: ', min_output)


i:  5
j:  121537743
seed:  1799914545
output:  78784898


In [125]:
# The step size used for the range that contains this approximate minimum
math.floor(math.sqrt(len(seed_list[5])))

11441

In [126]:
min_output = 999999999999999999999999
min_seed = 999999999999999999999999

# Fix the list that has the approximate minimum
i = 5

# Each the step size window around the approximate minimum
for j in range(121537743 - 11441, 121537743 + 11441):
    input = seed_list[i][j]

    for x in [x for x in data_dict.keys() if x != 'seeds']:
        output = data_map(input, data_dict[x])
        input = output

    if output < min_output:
        min_i = i
        min_j = j
        min_seed = seed_list[i][j]
        min_output = output

print('i: ', min_i)
print('j: ', min_j)
print('seed: ', min_seed)
print('output: ', min_output)

i:  5
j:  121527896
seed:  1799904698
output:  78775051


### Day 6

In [77]:
f = open('../data/day_6_input.txt', 'r')

data_dict = {}

for line in f:
    print(line)
    data = line.lower().replace(':', '').split()
    data_dict[data[0]] = [int(x) for x in data[1:]]

data_dict

Time:        41     66     72     66

Distance:   244   1047   1228   1040


{'time': [41, 66, 72, 66], 'distance': [244, 1047, 1228, 1040]}

In [50]:
counter_prod = 1

for race in range(len(data_dict['time'])):
    counter = 0
    for choice in range(data_dict['time'][race]+1):
        if choice*(data_dict['time'][race]-choice) > data_dict['distance'][race]:
            counter = counter + 1
            
    print(counter)
    counter_prod = counter_prod*counter

print(counter_prod)

26
13
17
13
74698


In [86]:
ms = ''
for i in range(len(data_dict['time'])):
    ms = ms+str(data_dict['time'][i])

ms = int(ms)

dist = ''
for i in range(len(data_dict['distance'])):
    dist = dist + str(data_dict['distance'][i])

dist = int(dist)

root_1 = (ms + math.sqrt(ms**2 - 4*dist))/2
root_2 = (ms - math.sqrt(ms**2 - 4*dist))/2

len(range(math.ceil(root_2), math.ceil(root_1)))


27563421

### Day 7

In [146]:
hand_value_map = {
    (5, None):(15**3)*6,
    (4, 1):(15**3)*5,
    (3, 2):(15**3)*4,
    (3, 1):(15**3)*3,
    (2, 2):(15**3)*2,
    (2, 1):(15**3)*1,
    (1, 1):(15**3)*0
}

card_map = {
    '2':2,
    '3':3,
    '4':4,
    '5':5,
    '6':6,
    '7':7,
    '8':8,
    '9':9,
    '10':10,
    'T':11,
    'J':12,
    'Q':13,
    'K':14,
    'A':15
}

card_map_jokers = {
    'J':1,
    '2':2,
    '3':3,
    '4':4,
    '5':5,
    '6':6,
    '7':7,
    '8':8,
    '9':9,
    '10':10,
    'T':11,
    'Q':12,
    'K':13,
    'A':14
}

def hand_value(hand:str, joker = False):
    if joker == True:
        hand = [card_map_jokers[x] for x in list(hand)]
    else:
        hand = [card_map[x] for x in list(hand)]

    hand_grouped = {x:hand.count(x) for x in hand}

    if joker == True:
        wilds = hand_grouped.pop(1, 0)
        
    values_sorted = sorted(hand_grouped.values(), reverse=True)
    if len(values_sorted) > 0:
        largest_group = values_sorted[0]
    else:
        largest_group = 0
    
    if joker == True:
        largest_group = largest_group + wilds

    if largest_group == 5:
        second_largest_group = None
    else:
        second_largest_group = values_sorted[1]

    value = hand_value_map[(largest_group, second_largest_group)]

    tiebreaker = 0
    for i in range(5):
        tiebreaker = tiebreaker + (hand[i]-1)/15**(i-2)

    return value + tiebreaker

In [144]:
f = open('../data/day_7_input.txt', 'r')

data_dict = {}

i = 0
for line in f:
    data = line.split()
    data_dict.update({i:{}})
    data_dict[i]['hand'] = data[0]
    data_dict[i]['bid'] = int(data[1])
    i = i + 1

df = pd.DataFrame.from_dict(data_dict, orient='index')

part_1 = (df
    .assign(value = df['hand'].apply(hand_value))
    .assign(rank = lambda x: x['value'].rank())
)

sum(part_1['rank']*part_1['bid'])

246912307.0

In [147]:
part_2 = (df
    .assign(value = df['hand'].apply(hand_value, joker=True))
    .assign(rank = lambda x: x['value'].rank())
)

sum(part_2['rank']*part_2['bid'])

246894760.0